# Finding out why Monte Carlo results are significantly different than static ones

Based on a [Stack Overflow question](https://stackoverflow.com/questions/72807629/overestimated-monte-carlo-results-in-brightway).

Uses Brightway 2.5.

In [ ]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba
import bw2io as bi
import numpy as np
import pandas as pd

This is setup for the DdS infrastructure and courses where we have access to ecoinvent. Adjust as needed.

In [ ]:
if "ei38-conseq-25" not in bd.projects:
    bi.restore_project_directory("/srv/data/projects/ecoinvent38-conseq.tar.gz")

In [ ]:
bd.projects.set_current("ei38-conseq-25")

Define functional unit and LCIA impact category

In [ ]:
waste_paper = bd.get_node(name='market for waste paper, sorted', database='ei 3.8 conseq')
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

Create static and stochastic LCA instances

In [ ]:
static = bc.LCA({waste_paper: 1}, ipcc)
static.lci()
static.lcia()

In [ ]:
mc = bc.LCA({waste_paper: 1}, ipcc, use_distributions=True)
mc.lci()
mc.lcia()

Confirm that the scores are quite different

In [ ]:
static.score, np.mean([mc.score for _ in zip(range(10), mc)])

We can do a recursive calculation on the Monte Carlo object (this doesn't call `__next__`, so the matrices remain the same), but we need to give some normally private arguments:

In [ ]:
ba.print_recursive_calculation(
    waste_paper,
    ipcc,
    amount=1,
    max_level=5,
    cutoff=0.01,
    _lca_obj=mc,
    _total_score=mc.score,
)

We can see from this that there is a large impact from `tissue paper production, virgin`. Let's switch our functional unit to this:

In [ ]:
tissue = bd.get_node(name='tissue paper production, virgin', database='ei 3.8 conseq')

But when we try to look through the supply chain to understand why we get such high stochastic impacts, we don't get anywhere:

In [ ]:
static = bc.LCA({tissue: 1}, ipcc)
static.lci()
static.lcia()

In [ ]:
mc = bc.LCA({tissue: 1}, ipcc, use_distributions=True)
mc.lci()
mc.lcia()

In [ ]:
ba.print_recursive_calculation(
    tissue,
    ipcc,
    amount=1,
    max_level=5,
    cutoff=0.025,
    _lca_obj=mc,
    _total_score=mc.score,
)

We seem to be missing something important - the total (stochastic) score is > 200, but the only input above our 2.5% cutoff is sulfite pulp, which is only 3.6% of the total score (specific numbers can change per MC iteration). Maybe this activity has some high direct emissions?

In [ ]:
mc.characterized_inventory[:, mc.dicts.activity[tissue.id]].sum()

We don't see anything in our supply chain...

[Something is hapening here, but you don't know what it is](https://www.youtube.com/watch?v=we37yX3zpKA)...

Not going to lie, this was very confusing. There are no direct emissions from tissue paper, but also no impacts from the supply chain of tissue paper. Eventually I realized that we can compare the values in the technosphere matrix for the static and Monte Carlo cases and see how different the input amounts are:

In [ ]:
def compare_inputs_two_lca_objects(first, second, column_id):
    column_index = first.dicts.activity[column_id]
    results = []
    
    for row in first.technosphere_matrix[:, column_index].tocoo().row:
        a = abs(first.technosphere_matrix[row, column_index])
        b = abs(second.technosphere_matrix[row, column_index])
        try:
            ratio = a / b
        except ZeroDivisionError:
            ratio = 0
        if ratio > 10 or ratio < 0.1:
            act = bd.get_activity(first.dicts.product.reversed[row])
            results.append({
                'ratio': ratio, 
                'a_value': a,
                'b_value': b,
                'name': act['name'],
                'location': act['name'],
                'act_id': act.id,
            })
            
    if results:
        results.sort(key=lambda x: x['ratio'], reverse=True)
        results = pd.DataFrame(results)
    return results

In [ ]:
compare_inputs_two_lca_objects(mc, static, tissue.id)

We are finally getting somewhere! We have four inputs to the tissue paper where the static amounts are **much less** than the stochastic values sampled. Let's look at one of the uncertainty distributions ([here is the activity dataset at ecoinvent](https://v38.ecoquery.ecoinvent.org/Details/UPR/62960c80-c028-44d4-9d45-a4a6a7a689c6/dd7f13f5-0658-489c-a19c-f2ff8a00bdd9)):

In [ ]:
exc = next(exc for exc in tissue.technosphere() if exc.input.id == 8907)
dist = {attr: exc.get(attr) for attr in ('uncertainty type', 'minimum', 'loc', 'maximum')}
dist

`5` is the [triangular distribution](https://en.wikipedia.org/wiki/Triangular_distribution), this comes from [stats_arrays](https://stats-arrays.readthedocs.io/en/latest/).

So, what does this mean? Let's look at the statistics:

In [ ]:
import stats_arrays as sa

In [ ]:
sa.TriangularUncertainty.statistics(sa.TriangularUncertainty.from_dicts(dist))

When the mean is more than 200 times the static value used, it makes sense that the Monte Carlo results would be significantly biased. And we see a similar pattern in the other three exchanges with high ratios.

Why didn't we see this in our recursive graph traversal? Because that uses the static exchange values instead of the values in the matrix itself:

```python
for exc in activity.technosphere():
```

As of `bw2analyzer` version 0.11.4, we have a modified `print_recursive_calculation` function which will use the actual matrix values (i.e. Monte Carlo samples) instead:

In [ ]:
ba.print_recursive_calculation(
    tissue,
    ipcc,
    amount=1,
    max_level=5,
    cutoff=0.025,
    use_matrix_values=True,
    _lca_obj=mc,
    _total_score=mc.score,
)

We can now find interesting results more quickly.